# 📘 Edge AI Model Dataset Builder
This notebook benchmarks multiple AI models (image, audio, NLP) on your device, builds a latency/memory dataset, trains a surrogate model, and selects the optimal model.

In [ ]:
!pip install psutil pynvml onnxruntime torch torchvision numpy pandas scikit-learn transformers requests

## 🧩 Step 1: Probe Device

In [ ]:
from edge_selector.probe import DeviceProbe
import json
profile = DeviceProbe().run()
json.dump(profile, open('device_profile.json','w'), indent=2)
print('✅ Device profile saved!')

## ⚙️ Step 2: Benchmark All Registered Models

In [ ]:
!python scripts/run_all_models.py

## 📊 Step 3: Inspect Dataset

In [ ]:
import pandas as pd, json
rows = [json.loads(l) for l in open('runs.ndjson')]
df = pd.DataFrame(rows)
display(df[['meta','framework','latency_ms','mem_mb']].head())
df.describe()

## 🧠 Step 4: Train Surrogate Model

In [ ]:
from edge_selector.surrogate import Surrogate
s = Surrogate()
s.train('runs.ndjson')
print('✅ Surrogate trained!')

## 🧮 Step 5: Select Best Model Based on SLA

In [ ]:
import json
from edge_selector.selector import Selector
context = json.load(open('device_profile.json'))
actions = [{'model': m, 'params': 1e6, 'backend': 'torch', 'est_quality': 0.85} for m in df['meta'].apply(lambda x: x['model_name']).unique()]
sel = Selector(s)
print(sel.select(context, actions))

## 📈 Step 6: Visualize Model Trade-offs

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(df['latency_ms'], df['mem_mb'])
plt.xlabel('Latency (ms)')
plt.ylabel('Memory (MB)')
plt.title('Model Latency vs Memory Usage')
for i,m in enumerate(df['meta'].apply(lambda x: x['model_name'])):
    plt.text(df['latency_ms'].iloc[i]+1, df['mem_mb'].iloc[i], m, fontsize=8)
plt.show()